In [1]:
## Import the required libraries and read the data into memory
## Rename the column names from spaces to underscores

import pandas as pd

df = pd.read_csv('./data/Trade_DetailedTradeMatrix.csv', encoding='latin-1')

colnames = []
for i in df.columns:
    i = i.replace(" ", "_")
    colnames.append(i)

df.columns = colnames

df.columns

df['Reporter_Country_Code_(M49)'] = df['Reporter_Country_Code_(M49)'].str.replace("'", "")
codes = pd.read_csv("./data/country_codes.csv")
codes = codes[["Country or Area", "ISO-alpha3 Code"]]
codes

,Country or Area,ISO-alpha3 Code
0,Algeria,DZA
1,Egypt,EGY
2,Libya,LBY
3,Morocco,MAR
4,Sudan,SDN
...,...,...
244,Samoa,WSM
245,Tokelau,TKL
246,Tonga,TON
247,Tuvalu,TUV


In [2]:
import duckdb

## This function calculates the dependency of a country on another country for a certain good
## For example: using dependency('United Arab Emirates', 2019, 'Rice') will give you the top
## countries that the United Arab Emirates depends on for rice in the year 2019.

def product_dependency(country, year, item):
    query = '''
    select Partner_Countries,
        sum(Value) /
        (
            select
                sum(Value)
            from df
            where Year = {year}
            and Reporter_Countries = '{country}'
            and Item = '{item}'
            and Element = 'Import Quantity'
        ) as dep_ratio
    from df
    where Year = {year}
    and Reporter_Countries = '{country}'
    and Element = 'Import Quantity'
    and Item = '{item}'
    and Value > 0
    group by Partner_Countries
    order by dep_ratio desc; 
'''.format(item = item, country = country, year = year)
    
    return duckdb.sql(query).fetchdf()

product_dependency('United Arab Emirates', 2019, 'Rice')

,Partner_Countries,dep_ratio
0,Pakistan,0.573980
1,India,0.364404
2,United States of America,0.026558
3,Thailand,0.011904
4,Philippines,0.007916
5,Sri Lanka,0.007144
6,Italy,0.003958
7,United Kingdom of Great Britain and Northern I...,0.002533
8,Belgium,0.000821
9,"China, mainland",0.000317


In [3]:
## This function calculates the trade dependency of a country by year by total import value.
## It fetches the top 20 countries by import value that the selected country imports from.
## For example, if you use trade_dependency('United Arab Emirates', 2021)
## You will get the top 20 countries by import value (USD $) that the U.A.E imports from
## In the year 2021. 

def trade_dependency(country, year):
    
    dataframe = duckdb.sql('''
    select Partner_Countries,
    sum(Value) as import_value,
    import_value /
        (select
            sum(Value)
            from df
            where Year = {year}
            and Element = 'Import Value'
            and Reporter_Countries = '{country}'
        ) as imp_prop
    from df
    where Year = {year}
    and Element = 'Import Value'
    and Reporter_Countries = '{country}'
    group by Partner_Countries
    having imp_prop > 0
    order by imp_prop desc;
    '''.format(year = year, country = country)).fetchdf().head(21)

    return dataframe

trade_dependency('Oman', 2018)

,Partner_Countries,import_value,imp_prop
0,United Arab Emirates,1824275.0,0.453535
1,India,397051.0,0.098711
2,Saudi Arabia,240000.0,0.059667
3,Pakistan,183415.0,0.045599
4,Somalia,141632.0,0.035211
5,Russian Federation,97926.0,0.024345
6,Indonesia,86039.0,0.021390
7,Iran (Islamic Republic of),76016.0,0.018898
8,Brazil,68755.0,0.017093
9,Argentina,62299.0,0.015488


In [4]:
## This function calculates how much leverage a country has over another country in terms of a specific product
## For example: using product_leverage('India', 2019, 'Rice')
## Will give you how much leverage India has over other countries in rice exports


def product_leverage(country, year, item):
    query = '''
        drop table if exists exp;
        drop table if exists imp;

        create table exp as
        select Reporter_Countries as reporting,
                    sum(Value) as trade
                from df
                where Year = {year}
                and Partner_Countries = '{country}'
                and Element = 'Import Quantity'
                and Item = '{item}'
                and Value > 0
                group by Reporter_Countries
                order by trade desc;

        create table imp as
        select Reporter_Countries,
                    sum(Value) as import_quantity
                from df
                where Year = {year}
                and Element = 'Import Quantity'
                and Item = '{item}'
                and Value > 0
                group by Reporter_Countries
                order by import_quantity desc;

        select Reporter_Countries, trade, import_quantity, trade / import_quantity as dep
        from exp
        left join imp
        on imp.Reporter_Countries = exp.reporting
        where dep between 0 and 1
        order by dep desc;
        '''.format(year = year, country = country, item = item)
    out = duckdb.query(query)

    return out.fetchdf()

product_leverage('India', 2019, 'Rice')

,Reporter_Countries,trade,import_quantity,dep
0,Bhutan,76.00,76.00,1.000000
1,Qatar,51.15,51.15,1.000000
2,Nepal,265475.00,265486.18,0.999958
3,Bosnia and Herzegovina,23.00,23.95,0.960334
4,Azerbaijan,3315.69,3459.85,0.958333
5,Seychelles,12.55,13.23,0.948602
6,Saudi Arabia,10483.66,11774.76,0.890350
7,Bangladesh,4430.02,5101.79,0.868327
8,Mongolia,10.00,12.49,0.800641
9,Kuwait,433.19,556.91,0.777846


In [35]:
## This function finds the trade dependants by value (USD $ of a country).
## For example, using trade_dependants('United Arab Emirates', 2021)
## Finds the top 20 countries that depend on the U.A.E
## The dependency for country A on country B is quantified by taking
## The proportion of imports from country B to country A over the total
## Imports to country A:
##
## dep_ratio = (Import value from B to A) / (Total import value to country A)

def trade_leverage(country, year):

    query = '''
    drop table if exists top_imports;
    drop table if exists total_imports;

    create table top_imports as
    select Reporter_Countries,
        sum(Value) as imports
        from df
        where Year = {year}
        and Element = 'Import Value'
        and Partner_Countries = '{country}'
        group by Reporter_Countries
        having imports > 0
        order by imports desc;

    create table total_imports as
    select Reporter_Countries,
        sum(Value) as import_value
        from df
        where Year = {year}
        and Element = 'Import Value'
        group by Reporter_Countries
        having import_value > 0
        order by import_value desc;
    
        drop table if exists imps;
        drop table if exists dependants;

        create table imps as
            select * from top_imports
            left join total_imports
            on top_imports.Reporter_Countries = total_imports.Reporter_Countries
            order by import_value desc;

            select Reporter_Countries, imports,
                imports / import_value as dep_ratio
                from imps
                order by dep_ratio desc;
        '''.format(year = year, country = country)

    dependants = duckdb.sql(query).fetchdf()
    
    return dependants

trade_leverage('Russian Federation', 2018)

import plotly.express as px

dt = product_leverage('India', 2019, 'Rice').join(codes.set_index("Country or Area"), on="Reporter_Countries")


fig = px.choropleth(dt, locations="ISO-alpha3 Code",
                    color="dep", 
                    hover_name="Reporter_Countries", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Blues)

fig.update_geos(
    visible=True,
    showcountries=True,
    countrycolor="black",
    projection_type="orthographic")

fig.update_layout(height=800)

fig.show()

SyntaxError: invalid syntax (1611104217.py, line 72)